In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
df.reset_index(inplace=True)

In [44]:
year = 2021
month = 2
df['ride_id']=f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [45]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [46]:
y_pred.mean()

16.191691679979066

In [47]:
y_pred.shape

(990113,)

In [56]:
df_result = pd.concat([df[['ride_id']], pd.Series(y_pred, name='pred')],axis=1)

In [57]:
df_result.shape

(990113, 2)

In [58]:
df_result.head()

,ride_id,pred
0,2021/02_0,14.539865
1,2021/02_1,13.740422
2,2021/02_2,15.593339
3,2021/02_3,15.188118
4,2021/02_4,13.817206


In [59]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990113 entries, 0 to 990112
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   ride_id  990113 non-null  object 
 1   pred     990113 non-null  float64
dtypes: float64(1), object(1)
memory usage: 15.1+ MB


In [60]:
df_result.to_parquet(
    "result.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)